# Task 1

## Original exercise number

Exercise 15-3

## Description

1. Write a definition for a type named `Circle` with fields `center` and `radius`, where `center` is a `Point` object and `radius` is a number.

2. Instantiate a circle object that represents a circle with its center at (150,100) and radius 75.

3. Write a function named `pointincircle` that takes a `Circle` object and a `Point` object and returns `true` if the point lies in or on the boundary of the circle.

4. Write a function named `rectincircle` that takes a `Circle` object and a `Rectangle` object and returns `true` if the rectangle lies entirely in or on the boundary of the circle.

5. Write a function named `rectcircleoverlap` that takes a `Circle` object and a `Rectangle` object and returns `true` if any of the corners of the rectangle fall inside the circle. Or as a more challenging version, return `true` if any part of the rectangle falls inside the circle.

## Solution

NO GUARANTEE THAT THE SOLUTION WILL WORK OR WORKS CORRECTLY! USE IT AT
YOUR OWN RISK!

### Imports

In [ ]:
# https://docs.julialang.org/en/v1/stdlib/Test/#Basic-Unit-Tests
import Test as ts

### Functions

In [ ]:
struct Point{T<:Real}
    x::T
    y::T
end

In [ ]:
function getDistance(p1::Point{<:Real}, p2::Point{<:Real})::Real
    dx::Real = p1.x - p2.x
    dy::Real = p1.y - p2.y
    return sqrt((dx^2) + (dy^2))
end

In [ ]:
struct Rectangle{T<:Real}
    width::T
    height::T
    lowerLeftCorner::Point{<:Real}
end

In [ ]:
struct Circle{T<:Real}
    center::Point{<:Real}
    radius::T
end

In [ ]:
struct Line{T<:Real}
    p1::Point{T}
    p2::Point{T}
end

In [ ]:
function isPointInCircle(p::Point{<:Real}, c::Circle{<:Real})::Bool
    distFromCenter::Real = getDistance(c.center, p)
    return distFromCenter <= c.radius
end

In [ ]:
# returns vertices, llc, urc, lrc, ulc
function get4vertices(rect::Rectangle{<:Real})::Vector{Point{<:Real}}
    llc, w, h = (rect.lowerLeftCorner, rect.width, rect.height)
    vertices::Vector{Point{<:Real}} = [llc]
    for (w1, h1) in zip([w, w, 0], [h, 0, h])
        push!(vertices, Point(llc.x + w1, llc.y + h1))
    end
    return vertices
end

In [ ]:
function get4sides(rect::Rectangle{<:Real})
    llc, urc, lrc, ulc = get4vertices(rect)
    return [Line(llc, lrc), Line(llc, ulc), Line(urc, lrc), Line(urc, ulc)]
end

In [ ]:
function isRectInCirc(rect::Rectangle{<:Real}, circ::Circle{<:Real})::Bool
    for v in get4vertices(rect)
        if !isPointInCircle(v, circ)
            return false
        end
    end
    return true
end

In [ ]:
# https://www.geeksforgeeks.org/check-line-touches-intersects-circle/

# a is the slope
function getA(lin::Line{<:Real})::Real
    p1, p2 = lin.p1, lin.p2
    slope::Real = (p2.y - p1.y) / (p2.x - p1.x)
    if slope == 0
        return 1e-16
    elseif slope == Inf
        return 1e16
    else
        return slope
    end
    # return slope
end

# c is the intercept
function getC(lin::Line{<:Real})::Real
    p1, p2 = lin.p1, lin.p2
    meanY = (p1.y + p2.y) / 2
    meanX = (p1.x + p2.x) / 2
    return meanY - getA(lin) * meanX
end

# changing: y = a*x + c to: a*x + b*y + c = 0
function getABC(lin::Line{<:Real})::Tuple{Real, Real, Real}
    a = getA(lin)
    c = getC(lin)
    # e.g. y = 2x + 1 => y - 2x - 1 = 0 (change sides and signs)
    return (-1*a, 1, -1*c)
end

function doesLineIntersectsCircle(lin::Line{<:Real}, circ::Circle{<:Real})::Bool
    a, b, c = getABC(lin)
    cx, cy = circ.center.x, circ.center.y
     
    # distance of line from circle center
    dist::Real = ((abs(a * cx + b * cy + c)) / sqrt(a * a + b * b))

    return circ.radius > dist
end

function doesAnyRectSideIntersectsCircle(rect::Rectangle{<:Real}, circ::Circle{<:Real})::Bool
    sides::Vector{Line{<:Real}} = get4sides(rect)
    intersections::Vector{Bool} = map(s -> doesLineIntersectsCircle(s, circ), sides)
    return any(intersections)
end

function doesRectOverlapCirc(rect::Rectangle{<:Real}, circ::Circle{<:Real})::Bool
    return isRectInCirc(rect, circ) || doesAnyRectSideIntersectsCircle(rect, circ)
end

## Testing

In [ ]:
circ1 = Circle(Point(150, 100), 75)
point1 = Point(100, 100)
point2 = Point(300, 300)

In [ ]:
ts.@test isPointInCircle(point1, circ1)

In [ ]:
ts.@test !isPointInCircle(point2, circ1)

In [ ]:
rect1 = Rectangle(75, 50, Point(100, 75))
rect2 = Rectangle(75, 50, Point(50, 100))

In [ ]:
ts.@test isRectInCirc(rect1, circ1)

In [ ]:
ts.@test !isRectInCirc(rect2, circ1)

In [ ]:
circ2 = Circle(Point(100, 100), 100)
rect3 = Rectangle(100, 50, Point(0, 0))
rect4 = Rectangle(50, 50, Point(-75, 75))

In [ ]:
ts.@test doesRectOverlapCirc(rect3, circ2)

In [ ]:
# test fails, check the solution
ts.@test !doesRectOverlapCirc(rect4, circ2)